In [1]:
from joblib import Parallel, delayed
import statsmodels.stats.power as smp
from scipy.stats import ttest_1samp
import pandas as pd
import numpy as np
import os

from util.preprocessing import calc_overest_means, huber_mean
from util.io import load_osf_binding_dataset
from util.simulation import DataSimulator

In [2]:
alpha_main = 0.01
alpha_inter = 0.05
p_aware = 0.05
n_simulations = 10000
cohens_d = 0.451 # meta-analytic effect size DOI:10.1163/22134468-20191150

In [3]:
# load n = 192 intentional binding dataset from OSF
# reported in https://doi.org/10.31234/osf.io/4z2rj
df = load_osf_binding_dataset() 

0files [00:00, ?files/s]
  0%|                                           | 0.00/4.06M [00:00<?, ?bytes/s]
  5%|█▋                                 | 197k/4.06M [00:00<00:01, 1.94Mbytes/s]
100%|██████████████████████████████████| 4.06M/4.06M [00:00<00:00, 18.2Mbytes/s]
1files [00:05,  5.54s/files]


In [4]:
# compute between and within subject variance estimates from OSF dataset
sds = []
operant_means = []
operant_means = []
baseline_means = []
for sub in df.subject.unique():
    df_sub = df[(df.subject == sub)]
    baseline = df_sub.overest_ms[df_sub.operant == 0]
    operant = df_sub.overest_ms[df_sub.operant == 1]
    sds.append(baseline.std())
    operant_means.append(huber_mean(operant))
    baseline_means.append(huber_mean(baseline))
baseline = np.array(baseline_means)
operant = np.array(operant_means)
diffs = (operant - baseline)
sds = np.array(sds)

sd_trial = np.mean(sds)
sd_block = sd_trial / np.sqrt(40)
sd_wi = np.sqrt(2*(sd_block**2))
sd_total = np.std(diffs)
sd_bw = np.sqrt(sd_total**2 - sd_wi**2)
frac_wi = (sd_wi**2) / (sd_total**2)
mu = np.mean(diffs)
print('Mean difference (ms): ', mu)
print('Within-subject standard deviation (trial): ', sd_trial)
print('Within-subject standard deviation (block mean): ', sd_block)
print('Within-subject standard deviation (block mean): ', sd_wi)
print('Between-subject standard deviation: ', sd_bw)
print('%.02f%% of the variance is within-subject.'%(100*frac_wi))

Mean difference (ms):  25.669576797923014
Within-subject standard deviation (trial):  92.78735221092478
Within-subject standard deviation (block mean):  14.670968552139124
Within-subject standard deviation (block mean):  20.74788269958432
Between-subject standard deviation:  78.78551687863724
6.49% of the variance is within-subject.


In [5]:
## adjust Cohen's d for interaction by doubling within-subject variance
d_interaction = cohens_d / np.sqrt(1 + frac_wi)
print("Cohen's d for difference-in-differences is %.02f"%d_interaction)

Cohen's d for difference-in-differences is 0.44


(The above works because we assume effect size in the masked conditions is 0 if $H_0$ is true for the main effect, so the only additional variance in the interaction given $H_0$ is a doubling of the within-subject variance.)

In [6]:
# pick sample size to achieve target power for main effect
def sample_size_calculation(d, alpha):
    power_analysis = smp.TTestPower()
    sample_size = power_analysis.solve_power(
        effect_size = d, 
        power = 0.95, 
        alpha = alpha, 
        alternative = 'larger'
    )
    sample_size = np.round(sample_size).astype(int) 
    return sample_size
ss_main = sample_size_calculation(cohens_d, alpha_main)
print('Sample size needed for main effect:', ss_main)
ss_iter = sample_size_calculation(d_interaction, alpha_inter)
print('Sample size needed for interaction effect:', ss_iter)
sample_size = np.max([ss_main, ss_iter])
print('So sample size is:', ss_main)

Sample size needed for main effect: 80
Sample size needed for interaction effect: 58
So sample size is: 80


In [7]:
def simulate(p_aware, n_subjects, seed = None):
    '''
    Simulates data that is null for the masked conditions, and has a nonzero
    effect for the unmasked conditions.
    
    All the DataFrame junk going on inside DataSimulator makes this extremely slow
    (we're talking 40+ times longer than if we just did this in numpy),
    but this way we get to use the same functions we'll use to analyze the real data.
    '''
    # simulate data
    sim = DataSimulator( 
        n_trials = 40, # per block
        effect_size_ms = mu,
        sd_pop = sd_bw,
        sd_sub = sd_trial
    )
    rng = np.random.default_rng(seed)
    dfs = [sim.simulate_subject(p_aware, rng) for sub in range(n_subjects)]
    means = [calc_overest_means(df) for df in dfs]
    df = pd.DataFrame(means)
    # compute paired differences
    delta_masked = df['masked operant'] - df['masked baseline']
    delta_unmasked = df['unmasked operant'] - df['unmasked baseline']
    delta2 = delta_masked - delta_unmasked
    # test for main effect of interest
    res = ttest_1samp(delta_masked, popmean = 0, alternative = 'greater')
    reject = res.pvalue < alpha_main
    return reject

In [8]:
## find combined false positive rate by simulation
pfunc = delayed(simulate)
parallel = Parallel(n_jobs = -1, verbose = 1)
output = parallel(pfunc(p_aware, sample_size, i) for i in range(n_simulations))
output = np.array(output)
fpr = output.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 788 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1238 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 2438 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 3188 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 4038 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 4988 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 6038 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-1)]: Done 7188 tasks      | elapsed: 30.0min
[Parallel(n_jobs=-1)]: Done 8438 tasks      | elapsed: 35.1min
[Parallel(n_jobs=-1)]: Done 9788 tasks      | elapsed: 40.6min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000

In [9]:
print('The false positive is rate = %.02f%% at p_aware = %.02f...'%(100*fpr, p_aware))
print('Approximately %.02f%% due to statistical error,'%(100*alpha_main))
print('plus %.02f%% due to residual awareness.'%(100*(fpr - alpha_main)))

accuracy = .5*(1 - p_aware) + 1.*p_aware
print('\nTo be sure p_aware is less than %.02f,' \
    ' then accuracy must be less than %.02f%%.'%(p_aware, 100*accuracy))

The false positive is rate = 3.79% at p_aware = 0.05...
Approximately 1.00% due to statistical error,
plus 2.79% due to residual awareness.

To be sure p_aware is less than 0.05, then accuracy must be less than 52.50%.


The originally proposed analysis, using $t$-tests, was replaced following interim peer-review. Consequently, we performed the simulations below to verify that the new tests have sufficient statistical power:

In [10]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import norm
import warnings

def simulate_mixedfx(n_subjects, seed = None):

    # simulate data
    sim = DataSimulator( 
        n_trials = 40, # per block
        effect_size_ms = mu,
        sd_pop = sd_bw,
        sd_sub = sd_trial
    )
    rng = np.random.default_rng(seed)
    dfs = [sim.simulate_subject(0., rng) for sub in range(n_subjects)]
    [df.insert(0, 'subject', i) for i, df in enumerate(dfs)]
    df = pd.concat(dfs)
    
    # test for main and interaction effect with multilevel model
    mod = smf.mixedlm('overest_t ~ operant*masked', df, groups = df['subject'])
    warnings.filterwarnings('ignore')
    mdf = mod.fit()
    # and extract one-tailed p-values for preregistered comparisons
    z = mdf.tvalues # really Z-values
    p_binding = norm.sf(z['operant[T.True]']) # upper tailed
    p_interaction = norm.cdf(z['operant[T.True]:masked[T.True]']) # lower tailed

    # return whether nulls rejected at specified significance levels 
    return p_binding < alpha_main, p_interaction < alpha_inter

## power analysis by simulation for mixed effects model
pfunc = delayed(simulate_mixedfx)
parallel = Parallel(n_jobs = -1, verbose = 1)
output = parallel(pfunc(sample_size, i) for i in range(n_simulations))
rejections = np.array(output)
power = rejections.mean(0)
print('Power for main effect at alpha = %.03f is %.03f'%(alpha_main, power[0]))
print('Power for interaction effect at alpha = %.03f is %.03f'%(alpha_inter, power[1]))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 788 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 1238 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 2438 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 3188 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 4038 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done 4988 tasks      | elapsed: 28.5min
[Parallel(n_jobs=-1)]: Done 6038 tasks      | elapsed: 34.6min
[Parallel(n_jobs=-1)]: Done 7188 tasks      | elapsed: 41.2min
[Parallel(n_jobs=-1)]: Done 8438 tasks      | elapsed: 48.2min
[Parallel(n_jobs=-1)]: Done 9788 tasks      | elapsed: 55.8min


Power for main effect at alpha = 0.010 is 0.984
Power for interaction effect at alpha = 0.050 is 0.982


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 57.1min finished


In [11]:
def simulate_mixedfx_null(n_subjects, seed = None):

    # simulate null data with spurious aware trials
    sim = DataSimulator( 
        n_trials = 40, # per block
        effect_size_ms = mu,
        sd_pop = sd_bw,
        sd_sub = sd_trial
    )
    rng = np.random.default_rng(seed)
    dfs = [sim.simulate_subject(p_aware, rng) for sub in range(n_subjects)]
    [df.insert(0, 'subject', i) for i, df in enumerate(dfs)]
    df = pd.concat(dfs)
    
    # test for main effect with multilevel model
    data = df[df.masked] # only masked condition trials
    mod = smf.mixedlm('overest_t ~ operant', data, groups = data['subject'])
    warnings.filterwarnings('ignore')
    mdf = mod.fit()
    # and extract one-tailed p-value for preregistered comparison
    z = mdf.tvalues # actually Z-values
    p = norm.sf(z['operant[T.True]'])

    # return whether null rejected at specified significance levels 
    return p < alpha_main

## estimate effective false positive rate of mixed effects model
pfunc = delayed(simulate_mixedfx_null)
parallel = Parallel(n_jobs = -1, verbose = 1)
output = parallel(pfunc(sample_size, i) for i in range(n_simulations))
rejections = np.array(output)
fpr = rejections.mean()
print('False positive rate = %.03f'%fpr)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 788 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 1238 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 1788 tasks      | elapsed: 26.2min
[Parallel(n_jobs=-1)]: Done 2438 tasks      | elapsed: 35.9min
[Parallel(n_jobs=-1)]: Done 3188 tasks      | elapsed: 47.3min
[Parallel(n_jobs=-1)]: Done 4038 tasks      | elapsed: 59.9min
[Parallel(n_jobs=-1)]: Done 4988 tasks      | elapsed: 74.0min
[Parallel(n_jobs=-1)]: Done 6038 tasks      | elapsed: 89.5min
[Parallel(n_jobs=-1)]: Done 7188 tasks      | elapsed: 106.6min
[Parallel(n_jobs=-1)]: Done 8438 tasks      | elapsed: 125.3min
[Parallel(n_jobs=-1)]: Done 9788 tasks      | elapsed: 146.5min


False positive rate = 0.039


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 149.7min finished


In [12]:
print('The false positive is rate = %.02f%% at p_aware = %.02f...'%(100*fpr, p_aware))
print('Approximately %.02f%% due to statistical error,'%(100*alpha_main))
print('plus %.02f%% due to residual awareness.'%(100*(fpr - alpha_main)))

The false positive is rate = 3.87% at p_aware = 0.05...
Approximately 1.00% due to statistical error,
plus 2.87% due to residual awareness.
